# Tuning-In To NYC’s Most Musical Neighborhoods
##### by Michael Cascio

### Import Required Python Packages

In [63]:
import folium
import json
import math
import numpy as np
import pandas as pd
import requests
from pandas.io.json import json_normalize

### Set Foursquare API Credentials

In [64]:
CLIENT_ID = 'TTTYSUWFUMJXP00B4DT3JJU0RB4GZ5C25DIEK3EJ2QBMHWK0' # your Foursquare ID
CLIENT_SECRET = 'XU5LCC4PNFPR0JBA41A45ZHHHTAVTTNX1VJUZINIY1X2SF0E' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Import NYC Neighborhood Name & Location Centroid Data Hosted From NYU's Spatial Data Repository

In [65]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
nyc_neighborhood_data = newyork_data['features']
nyc_neighborhood_data[2]

{'type': 'Feature',
 'id': 'nyu_2451_34572.3',
 'geometry': {'type': 'Point',
  'coordinates': [-73.82780644716412, 40.887555677350775]},
 'geometry_name': 'geom',
 'properties': {'name': 'Eastchester',
  'stacked': 1,
  'annoline1': 'Eastchester',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.82780644716412,
   40.887555677350775,
   -73.82780644716412,
   40.887555677350775]}}

In [66]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

nyc_neighborhoods = pd.DataFrame(columns=column_names)

for data in nyc_neighborhood_data:
    borough = nyc_neighborhood_data = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    nyc_neighborhoods = nyc_neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
nyc_neighborhoods.head(5)

,Borough,Neighborhood,Latitude,Longitude,Radius
0,Bronx,Wakefield,40.894705,-73.847201,NaN
1,Bronx,Co-op City,40.874294,-73.829939,NaN
2,Bronx,Eastchester,40.887556,-73.827806,NaN
3,Bronx,Fieldston,40.895437,-73.905643,NaN
4,Bronx,Riverdale,40.890834,-73.912585,NaN


### Retrieve Music-Related Venue Data From Foursquare

#### Foursquare Music-Related Venue CategoryIDs

Music Venue = '4bf58dd8d48988d1e5931735'

Jazz Club = '4bf58dd8d48988d1e7931735'

Piano Bar = '4bf58dd8d48988d1e8931735'

Rock Club = '4bf58dd8d48988d1e9931735'

Concert Hall = '5032792091d4c4b30a586d5c'

Amphitheater = '56aa371be4b08b9a8d5734db'

Music Festival = '5267e4d9e4b0ec79466e48d1'

Nightlife Spot = '4d4b7105d754a06376d81259'

Music Store = '4bf58dd8d48988d1fe941735'

Record Shop = '4bf58dd8d48988d10d951735'

Recording Studio = '52f2ab2ebcbc57f1066b8b37'

#### Function To Create API Url

In [83]:
def createURL(endpoint, CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, categoryId, limit):
    url = '{}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&limit={}'.format(
            endpoint,
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            categoryId,
            limit
        )
    return url

endpoint = 'https://api.foursquare.com/v2/venues/search?'
categoryIds = ['4bf58dd8d48988d1e5931735', '4bf58dd8d48988d1e7931735', '4bf58dd8d48988d1e8931735',
               '4bf58dd8d48988d1e9931735', '5032792091d4c4b30a586d5c', '56aa371be4b08b9a8d5734db',
               '5267e4d9e4b0ec79466e48d1', '4d4b7105d754a06376d81259', '4bf58dd8d48988d1fe941735',
               '4bf58dd8d48988d10d951735', '52f2ab2ebcbc57f1066b8b37',
              ]
categoryId = ','.join(categoryIds)
limit = 50

#### Function To Retrieve All Music-Related Venues Per Neighborhood
(Currently commented out because the data has already been retrieved and saved to a csv file)

In [72]:
'''
def getNearbyMusicVenues(neighborhoods, latitudes, longitudes, radius=1000):
    endpoint = 'https://api.foursquare.com/v2/venues/search?'
    venues_list = []   
    
    for hood_name, lat, lng in zip(neighborhoods, latitudes, longitudes):
    
        url = createURL(endpoint, CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, categoryId, limit)
        
        results = requests.get(url).json()['response']['venues']

        for item in results:
            venue_name = item['name']
            venue_category = item['categories'][0]['name']
            venue_lat = item['location']['lat']
            venue_lng = item['location']['lng']
            try:
                venue_city = item['location']['city']
            except:
                venue_city = 'N/A'
            venue_state = item['location']['state']
            
            venues_list.append([(hood_name,
                                lat,
                                lng,
                                venue_name,
                                venue_category,
                                venue_lat,
                                venue_lng,
                                venue_city,
                                venue_state
                               )])
            
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                             'Neighborhood Latitude',
                             'Neighborhood Longitude',
                             'Venue Name',
                             'Venue Category',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue City',
                             'Venue State'
                            ]
    return nearby_venues

prelim_venue_data = getNearbyMusicVenues(nyc_neighborhoods['Neighborhood'],
                                         nyc_neighborhoods['Latitude'],
                                         nyc_neighborhoods['Longitude'],
                                         radius
                                        )

'''

#### How Many Venues Did We Retrieve?
(Currently commented out because the data has already been retrieved and saved to a csv file)

In [87]:
'''
print(prelim_venue_data.shape)
prelim_venue_data.head(5)
'''

(9442, 9)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Name,Venue Category,Venue Latitude,Venue Longitude,Venue City,Venue State
0,Wakefield,40.894705,-73.847201,Major League Music Group,Music Venue,40.891367,-73.850088,N/A,New York
1,Wakefield,40.894705,-73.847201,Dyme Life Radio,Music Venue,40.894541,-73.843266,Bronx,NY
2,Wakefield,40.894705,-73.847201,Par-City,Music Venue,40.890211,-73.847002,Bronx,NY
3,Wakefield,40.894705,-73.847201,The Upper Room,Music Venue,40.892567,-73.846406,N/A,New York
4,Wakefield,40.894705,-73.847201,Tavern,Bar,40.895898,-73.855731,Bronx,NY


#### Save DataFrame As CSV To Avoid Further API Calls
(Currently commented out because the data has already been retrieved and saved to a csv file)

In [85]:
'''prelim_venue_data.to_csv(path_or_buf='prelim_venue_data.csv')'''

#### Loading the previously saved DataFrame from csv

In [421]:
pelim_venue_data = pd.read_csv('prelim_venue_data.csv')
prelim_venue_data.head(5)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Name,Venue Category,Venue Latitude,Venue Longitude,Venue City,Venue State
0,Wakefield,40.894705,-73.847201,Major League Music Group,Music Venue,40.891367,-73.850088,N/A,New York
1,Wakefield,40.894705,-73.847201,Dyme Life Radio,Music Venue,40.894541,-73.843266,Bronx,NY
2,Wakefield,40.894705,-73.847201,Par-City,Music Venue,40.890211,-73.847002,Bronx,NY
3,Wakefield,40.894705,-73.847201,The Upper Room,Music Venue,40.892567,-73.846406,N/A,New York
4,Wakefield,40.894705,-73.847201,Tavern,Bar,40.895898,-73.855731,Bronx,NY


### Exploratory Data Analysis

#### What states are the venues in?

In [103]:
prelim_venue_data.groupby('Venue State')['Venue State'].count()

Venue State
CA             1
MA             4
NJ            15
NY          8786
New York     636
Name: Venue State, dtype: int64

#### What venue categories are the entries in?

In [102]:
n_unique = len(prelim_venue_data['Venue Category'].unique())
print(f'There are {n_unique} unique venue categories in this dataframe')
prelim_venue_data.groupby('Venue Category')['Venue Category'].count().sort_values(ascending=False)

There are 149 unique venue categories in this dataframe


Venue Category
Bar                                         2124
Lounge                                       901
Cocktail Bar                                 492
Music Venue                                  477
Nightclub                                    474
Other Nightlife                              469
Pub                                          365
Wine Bar                                     277
Sports Bar                                   276
Hookah Bar                                   255
Beer Garden                                  218
Dive Bar                                     214
Karaoke Bar                                  178
American Restaurant                          169
Concert Hall                                 142
Speakeasy                                    131
Nightlife Spot                               120
Brewery                                      119
Record Shop                                  119
Rock Club                                    114
Music

#### How many venues did not have their 'city' parameter filled-out?

In [113]:
n_unknown_cities = prelim_venue_data[prelim_venue_data['Venue City']=='N/A']['Venue Name'].count()
print(f'There were {n_unknown_cities} entries in the dataset without the "city" parameter filled-out')

There were 636 entries in the dataset without the "city" parameter filled-out


In [116]:
prelim_venue_data[prelim_venue_data['Venue City']=='N/A']['Venue Name'][0:5]

0     Major League Music Group
3               The Upper Room
17                 Boohran Pub
22                   MY Studio
30                 Expressions
Name: Venue Name, dtype: object

#### Are there any 'null' values in the dataset?

In [117]:
prelim_venue_data.isnull().values.any()

False

#### How many unique venues are there?

In [121]:
n_unique_venues = len(prelim_venue_data['Venue Name'].unique())
print(f'There are {n_unique_venues} unique venues in the preliminary dataset')

There are 5039 unique venues in the preliminary dataset


### Data Pre-Processing

#### Remove entries with "Venue State" not equal to "New York" or "NY"

In [151]:
ny_venue_data = prelim_venue_data[(prelim_venue_data['Venue State'] == "New York") | (prelim_venue_data['Venue State'] == "NY")]
ny_venue_data['Venue State'].replace(to_replace="New York", value="NY", inplace=True)
delta = prelim_venue_data.shape[0] - ny_venue_data.shape[0]
print(f'{delta} entries were removed from the preliminary dataset based on "Venue State"')
ny_venue_data.head(5)

20 entries were removed from the preliminary dataset based on "Venue State"


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Name,Venue Category,Venue Latitude,Venue Longitude,Venue City,Venue State
0,Wakefield,40.894705,-73.847201,Major League Music Group,Music Venue,40.891367,-73.850088,N/A,NY
1,Wakefield,40.894705,-73.847201,Dyme Life Radio,Music Venue,40.894541,-73.843266,Bronx,NY
2,Wakefield,40.894705,-73.847201,Par-City,Music Venue,40.890211,-73.847002,Bronx,NY
3,Wakefield,40.894705,-73.847201,The Upper Room,Music Venue,40.892567,-73.846406,N/A,NY
4,Wakefield,40.894705,-73.847201,Tavern,Bar,40.895898,-73.855731,Bronx,NY


#### Remove entries with "Venue City" equal to "N/A"

In [156]:
ny_venue_data_with_city = ny_venue_data[(ny_venue_data['Venue City'] != "N/A")]
delta = ny_venue_data.shape[0] - ny_venue_data_with_city.shape[0]
print(f'{delta} entries were removed based on "Venue City"')
ny_venue_data_with_city.head(5)

636 entries were removed based on "Venue City"


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Name,Venue Category,Venue Latitude,Venue Longitude,Venue City,Venue State
1,Wakefield,40.894705,-73.847201,Dyme Life Radio,Music Venue,40.894541,-73.843266,Bronx,NY
2,Wakefield,40.894705,-73.847201,Par-City,Music Venue,40.890211,-73.847002,Bronx,NY
4,Wakefield,40.894705,-73.847201,Tavern,Bar,40.895898,-73.855731,Bronx,NY
5,Wakefield,40.894705,-73.847201,M.O.S. Cigars,Smoke Shop,40.890628,-73.848478,Bronx,NY
6,Wakefield,40.894705,-73.847201,Inn At the Border,Other Nightlife,40.898248,-73.838641,Mount Vernon,NY


#### Remove entries that are not associated with music-related categories

In [186]:
music_related_categories = ['Music Venue', 'Lounge', 'Nightclub', 'Jazz Club', 'Recording Studio', 'Piano Bar',
                            'Record Shop', 'Concert Hall', 'Karaoke Bar', 'Music Store', 'Rock Club', 'Amphitheater',
                            'Music Festival', 'Opera House', 'Music School']
ny_music_venues = ny_venue_data_with_city[ny_venue_data_with_city['Venue Category'].isin(music_related_categories)]
delta = ny_venue_data_with_city.shape[0] - ny_music_venues.shape[0]
print(f'{delta} entries were removed based on "Venue Category" not being related to music')
print(ny_music_venues['Venue Category'].unique())
ny_music_venues.head(5)

6401 entries were removed based on "Venue Category" not being related to music
['Music Venue' 'Lounge' 'Nightclub' 'Jazz Club' 'Recording Studio'
 'Piano Bar' 'Record Shop' 'Concert Hall' 'Karaoke Bar' 'Music Store'
 'Rock Club' 'Amphitheater' 'Music Festival' 'Opera House' 'Music School']


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Name,Venue Category,Venue Latitude,Venue Longitude,Venue City,Venue State
1,Wakefield,40.894705,-73.847201,Dyme Life Radio,Music Venue,40.894541,-73.843266,Bronx,NY
2,Wakefield,40.894705,-73.847201,Par-City,Music Venue,40.890211,-73.847002,Bronx,NY
12,Wakefield,40.894705,-73.847201,Emma C. Brisbane Youth Leadership Center,Lounge,40.887923,-73.853688,Bronx,NY
13,Wakefield,40.894705,-73.847201,Karma Lounge,Nightclub,40.901605,-73.850720,Bronx,NY
14,Wakefield,40.894705,-73.847201,Wenbley Lounge,Lounge,40.897526,-73.859468,Bronx,NY


#### How many music venue entries are left?

In [187]:
n_entries = ny_music_venues.shape[0]
print(f'There are {n_entries} entries in the ny_music_venues dataframe')

There are 2385 entries in the ny_music_venues dataframe


#### How many unique music venues are left?

In [188]:
n_unique_entries = len(ny_music_venues['Venue Name'].unique())
print(f'There are {n_unique_entries} unique entries in the ny_music_venues dataframe')

There are 1320 unique entries in the ny_music_venues dataframe


#### One-Hot-Encode Venue Categories

In [226]:
ny_venue_category_onehot = pd.get_dummies(ny_music_venues[['Venue Category']], prefix="", prefix_sep="")

ny_venue_category_onehot['Neighborhood'] = ny_music_venues['Neighborhood'] 

fixed_columns = [ny_venue_category_onehot.columns[-1]] + list(ny_venue_category_onehot.columns[:-1])
ny_venue_category_onehot = ny_venue_category_onehot[fixed_columns]

print(ny_venue_category_onehot.shape)
ny_venue_category_onehot.head(10)

(2385, 16)


,Neighborhood,Amphitheater,Concert Hall,Jazz Club,Karaoke Bar,Lounge,Music Festival,Music School,Music Store,Music Venue,Nightclub,Opera House,Piano Bar,Record Shop,Recording Studio,Rock Club
1,Wakefield,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,Wakefield,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
12,Wakefield,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
13,Wakefield,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
14,Wakefield,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
18,Wakefield,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
21,Wakefield,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
23,Wakefield,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
24,Co-op City,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
29,Co-op City,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


#### Determine the total amount of venues of each category in each neighborhood

In [276]:
venue_counts = ny_venue_category_onehot.groupby('Neighborhood').sum()
venue_counts.head(10)

,Amphitheater,Concert Hall,Jazz Club,Karaoke Bar,Lounge,Music Festival,Music School,Music Store,Music Venue,Nightclub,Opera House,Piano Bar,Record Shop,Recording Studio,Rock Club
Neighborhood,,,,,,,,,,,,,,,
Allerton,0,0,0,0,3,0,0,0,2,3,0,0,0,0,0
Arden Heights,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
Arrochar,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
Astoria,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
Astoria Heights,0,0,0,1,3,0,0,0,1,2,0,0,0,0,0
Auburndale,0,0,0,3,2,0,0,0,2,1,0,0,0,0,1
Bath Beach,0,0,0,1,9,0,0,1,1,1,0,0,0,0,1
Bay Ridge,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0
Bay Terrace,0,0,0,0,3,0,0,1,2,1,0,0,0,0,0


#### Determine the total amount of venues of each category

In [277]:
venue_totals = {}
for category in music_related_categories:
    venue_totals[category] = venue_counts[category].sum()

venue_totals

{'Music Venue': 431,
 'Lounge': 816,
 'Nightclub': 425,
 'Jazz Club': 47,
 'Recording Studio': 76,
 'Piano Bar': 23,
 'Record Shop': 100,
 'Concert Hall': 115,
 'Karaoke Bar': 138,
 'Music Store': 100,
 'Rock Club': 104,
 'Amphitheater': 3,
 'Music Festival': 1,
 'Opera House': 1,
 'Music School': 5}

#### For each venue category, determine the percentage of entities in each neighborhood

In [279]:
venue_mean = pd.DataFrame()
for category, total in venue_totals.items():
    venue_mean[category] = venue_counts[category].apply(lambda x: x / total)
venue_mean = venue_mean.reindex(sorted(venue_mean.columns), axis=1).reset_index()
venue_mean.head(5)

,Neighborhood,Amphitheater,Concert Hall,Jazz Club,Karaoke Bar,Lounge,Music Festival,Music School,Music Store,Music Venue,Nightclub,Opera House,Piano Bar,Record Shop,Recording Studio,Rock Club
0,Allerton,0.0,0.000000,0.0,0.000000,0.003676,0.0,0.0,0.0,0.00464,0.007059,0.0,0.0,0.00,0.0,0.0
1,Arden Heights,0.0,0.008696,0.0,0.000000,0.001225,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.00,0.0,0.0
2,Arrochar,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.002353,0.0,0.0,0.01,0.0,0.0
3,Astoria,0.0,0.000000,0.0,0.007246,0.001225,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.00,0.0,0.0
4,Astoria Heights,0.0,0.000000,0.0,0.007246,0.003676,0.0,0.0,0.0,0.00232,0.004706,0.0,0.0,0.00,0.0,0.0


#### Creating a dataframe that shows the top 5 venue categories each neighborhood is known for

In [258]:
def return_top_venue_categories(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [367]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Top Venue Category'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Top Venue Category'.format(ind+1))

neighborhoods_top_venue_categories = pd.DataFrame(columns=columns)
neighborhoods_top_venue_categories['Neighborhood'] = venue_mean['Neighborhood']

for ind in np.arange(venue_mean.shape[0]):
    neighborhoods_top_venue_categories.iloc[ind, 1:] = return_top_venue_categories(venue_mean.iloc[ind, :], num_top_venues)

neighborhoods_top_venue_categories.head(5)


,Neighborhood,1st Top Venue Category,2nd Top Venue Category,3rd Top Venue Category,4th Top Venue Category,5th Top Venue Category
0,Allerton,Nightclub,Music Venue,Lounge,Rock Club,Recording Studio
1,Arden Heights,Concert Hall,Lounge,Rock Club,Recording Studio,Record Shop
2,Arrochar,Record Shop,Nightclub,Rock Club,Recording Studio,Piano Bar
3,Astoria,Karaoke Bar,Lounge,Rock Club,Recording Studio,Record Shop
4,Astoria Heights,Karaoke Bar,Nightclub,Lounge,Music Venue,Rock Club


### Create KMeans Model To Cluster Neighborhoods

In [368]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# set number of clusters
kclusters = 15
scaler = StandardScaler()

venue_grouped_clustering = venue_mean.drop('Neighborhood', 1)
scaler.fit(venue_grouped_clustering)
scaled_venue_grouped_clustering = scaler.transform(venue_grouped_clustering)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scaled_venue_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([ 2, 12, 12, 12, 12, 12,  8, 12,  8, 12, 12, 14, 14,  8,  8, 12,  8,
        5,  8,  1, 12,  8, 11,  3,  1,  0, 12, 12,  1, 12, 12, 12,  2,  8,
        5, 12, 12,  2,  3, 12, 12,  6, 12, 11, 12,  2, 12,  0,  0,  0,  1,
       12, 12,  0, 14,  2, 13,  1,  8,  5,  8,  5, 12, 12, 12,  1,  8,  2,
       14, 11,  2,  2,  6, 12,  1,  2, 12, 12, 11, 14, 12,  5, 14, 12,  5,
       12,  5,  2, 14,  6,  7,  5,  2,  2, 12,  8,  5,  0, 12, 12, 12, 12,
       12, 12,  5, 12, 12, 12, 11,  8, 12, 12,  6,  6,  1, 12, 11, 14,  1,
       12,  2,  2, 11,  0,  2,  0, 10, 12, 14,  8, 14,  2, 14,  8,  8, 12,
        2,  2,  0,  1,  5, 12,  4,  0,  6, 12, 10,  1,  6,  3,  8,  3,  8,
        2,  3, 12,  5,  3, 12, 12,  8, 12, 12,  0,  5,  5, 12, 12,  3, 14,
        1,  8,  2, 14, 14,  7, 12,  8,  8, 12, 12, 11,  6,  2,  2, 12,  2,
       12, 12,  1,  3,  2, 11, 12,  0, 12, 11,  8,  1,  2, 12,  0,  0, 12,
        0, 12,  9, 11, 12, 12, 10,  0, 12,  5,  1, 14,  2, 12, 12, 12,  2,
       12, 12,  0, 12,  0

#### Creating a new dataframe merging neighborhood location data, top venue category by neighborhood, and cluster labels

In [369]:
neighborhoods_top_venue_categories.insert(1,'Cluster Labels', kmeans.labels_)

ny_neighborhood_music_profile = nyc_neighborhoods.drop(columns=['Borough', 'Radius'])

ny_neighborhood_music_profile = ny_neighborhood_music_profile.join(neighborhoods_top_venue_categories.set_index('Neighborhood'), on='Neighborhood').dropna(axis=0)

ny_neighborhood_music_profile.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Top Venue Category,2nd Top Venue Category,3rd Top Venue Category,4th Top Venue Category,5th Top Venue Category
0,Wakefield,40.894705,-73.847201,2.0,Music Venue,Nightclub,Lounge,Rock Club,Recording Studio
1,Co-op City,40.874294,-73.829939,1.0,Jazz Club,Recording Studio,Nightclub,Music Venue,Rock Club
2,Eastchester,40.887556,-73.827806,1.0,Recording Studio,Jazz Club,Nightclub,Music Venue,Rock Club
3,Fieldston,40.895437,-73.905643,5.0,Piano Bar,Record Shop,Music Venue,Lounge,Nightclub
4,Riverdale,40.890834,-73.912585,2.0,Music Venue,Nightclub,Lounge,Rock Club,Recording Studio


#### Creating A Visualization of The Clusters

In [373]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors
latitude = 40.730610
longitude = -73.935242
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10.45)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_neighborhood_music_profile['Latitude'], ny_neighborhood_music_profile['Longitude'], ny_neighborhood_music_profile['Neighborhood'], ny_neighborhood_music_profile['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examining Clusters

In [410]:
for cluster in range(0, kclusters):
    print(f'Cluster {cluster}:')
    print(ny_neighborhood_music_profile.loc[ny_neighborhood_music_profile['Cluster Labels'] == cluster, ny_neighborhood_music_profile.columns[[0] + list(range(4, ny_neighborhood_music_profile.shape[1]))]], end="\n\n\n")

Cluster 0:
        Neighborhood 1st Top Venue Category 2nd Top Venue Category  \
22       Port Morris           Concert Hall            Music Venue   
84      Clinton Hill       Recording Studio            Record Shop   
111  Upper West Side              Jazz Club           Concert Hall   
113          Clinton           Concert Hall            Music Venue   
114          Midtown           Concert Hall            Music Venue   
133     Howard Beach              Rock Club           Concert Hall   
158    Fresh Meadows            Music Store           Concert Hall   
164    South Jamaica            Music Store           Concert Hall   
187       Lindenwood           Concert Hall                 Lounge   
188        Laurelton           Concert Hall                 Lounge   
191    Rockaway Park           Concert Hall            Music Venue   
200         Rosebank           Concert Hall            Music Store   
216          Woodrow           Concert Hall              Rock Club   
224      